In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

In [2]:
import tensorflow as tf
import keras.backend.tensorflow_backend as KTF
 
config = tf.ConfigProto()  
config.gpu_options.allow_growth=True  
session = tf.Session(config=config)
 
KTF.set_session(session)

Using TensorFlow backend.


In [3]:
from preprocessing import *
from models import *
from evaluation import *
from train import *

In [4]:
data_root_path = '/data/qit16/FairML'
LabelWeights, FeatureDict, TrainFeatures, ValFeatures, TestFeatures, TrainAttri, ValAttri, TestAttri, TrainLabels, ValLabels, TestLabels = load_data(data_root_path)

In [ ]:
mode = 'Dense'
dim_rep = 64
hyper_gender = 500
hyper_age = 60
gamma = 0.1

for repeat in range(40):
    Result = []
    model,rep_model,Discrimator,Mapper,Contrastive_Discrimator,Contrastive_Attacker = get_data_flow(mode,FeatureDict,hyper_gender,hyper_age,gamma)
    for i in range(10):
        train_model(model,rep_model,Discrimator,Mapper,Contrastive_Discrimator,Contrastive_Attacker,LabelWeights,TrainFeatures,TrainAttri,TrainLabels)
        val_result = evaluate_val(model,rep_model,LabelWeights,TrainFeatures,TrainAttri,ValFeatures,ValLabels,ValAttri)
        test_result = evaluate_test(model,rep_model,LabelWeights,TestFeatures,TestLabels,TestAttri)
        privacy = evaluate_privacy(rep_model,Mapper,TestFeatures,FeatureDict)
        Result.append([test_result,val_result,privacy])
        print(Result[-1])

    dump_result(Result)
    print()